<a href="https://colab.research.google.com/github/juli275/IA_UdeA_2023_2/blob/main/Fake_Job_Posting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Carga de Librerias y Datos

In [45]:
import numpy as np
import pandas as pd

In [46]:
import nltk
from nltk.corpus import stopwords

In [47]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
url = 'https://media.githubusercontent.com/media/juli275/IA_UdeA_2023_2/main/fake_job_postings.csv'

In [43]:
Dataset = pd.read_csv(url)
Dataset01 = Dataset.copy()

#Visualizacion Datos de Kaggle

In [44]:
Dataset

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17875,17876,Account Director - Distribution,"CA, ON, Toronto",Sales,NaN,Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,What can you expect from us?We have an open cu...,0,1,1,Full-time,Mid-Senior level,NaN,Computer Software,Sales,0
17876,17877,Payroll Accountant,"US, PA, Philadelphia",Accounting,NaN,WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Health &amp; WellnessMedical planPrescription ...,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Internet,Accounting/Auditing,0
17877,17878,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",NaN,NaN,We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,NaN,0,0,0,Full-time,NaN,NaN,NaN,NaN,0
17878,17879,Graphic Designer,"NG, LA, Lagos",NaN,NaN,NaN,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Competitive salary (compensation will be based...,0,0,1,Contract,Not Applicable,Professional,Graphic Design,Design,0


**Exploración de datos**

Comenzaremos buscando aquellas características que tenga en 60% de los datos en null y las quitares de nuestro dataset

In [48]:
Dataset01.isnull().mean()<0.4

job_id                  True
title                   True
location                True
department             False
salary_range           False
company_profile         True
description             True
requirements            True
benefits               False
telecommuting           True
has_company_logo        True
has_questions           True
employment_type         True
required_experience     True
required_education     False
industry                True
function                True
fraudulent              True
dtype: bool

In [49]:
Dataset01= Dataset01[Dataset01.columns[Dataset01.isnull().mean()<0.4]]

Después de esto rellenaremos el null por un string vacío

In [50]:
Dataset01.fillna('',inplace=True)

Eliminaremos algunas columnas de nuestro dataset debido a que no son muy útiles para lo que deseamos realizar

In [51]:
Dataset01.drop(columns=['telecommuting','has_company_logo','has_questions','job_id'], inplace=True)

In [52]:
Dataset01

,title,location,company_profile,description,requirements,employment_type,required_experience,industry,function,fraudulent
0,Marketing Intern,"US, NY, New York","We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,Other,Internship,,Marketing,0
1,Customer Service - Cloud Video Production,"NZ, , Auckland","90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,Full-time,Not Applicable,Marketing and Advertising,Customer Service,0
2,Commissioning Machinery Assistant (CMA),"US, IA, Wever",Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,,,,0
3,Account Executive - Washington DC,"US, DC, Washington",Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Full-time,Mid-Senior level,Computer Software,Sales,0
4,Bill Review Manager,"US, FL, Fort Worth",SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full-time,Mid-Senior level,Hospital & Health Care,Health Care Provider,0
...,...,...,...,...,...,...,...,...,...,...
17875,Account Director - Distribution,"CA, ON, Toronto",Vend is looking for some awesome new talent to...,Just in case this is the first time you’ve vis...,To ace this role you:Will eat comprehensive St...,Full-time,Mid-Senior level,Computer Software,Sales,0
17876,Payroll Accountant,"US, PA, Philadelphia",WebLinc is the e-commerce platform and service...,The Payroll Accountant will focus primarily on...,- B.A. or B.S. in Accounting- Desire to have f...,Full-time,Mid-Senior level,Internet,Accounting/Auditing,0
17877,Project Cost Control Staff Engineer - Cost Con...,"US, TX, Houston",We Provide Full Time Permanent Positions for m...,Experienced Project Cost Control Staff Enginee...,At least 12 years professional experience.Abil...,Full-time,,,,0
17878,Graphic Designer,"NG, LA, Lagos",,Nemsia Studios is looking for an experienced v...,1. Must be fluent in the latest versions of Co...,Contract,Not Applicable,Graphic Design,Design,0


Para procesar el texto vamos a colocar todos las características juntas en una columna que llamaremos text. Y eliminaremos el resto de las columnas, de este modo tendremos todas las características en un mismo lugar

In [ ]:
Dataset01['text'] = Dataset01['title']+" "+Dataset01['location']+" "+Dataset01['company_profile']+" "+Dataset01['description']+" "+Dataset01['requirements']+" "+Dataset01['employment_type']+" "+Dataset01['required_experience']+" "+Dataset01['industry']+" "+Dataset01['function']

In [ ]:
Dataset01.drop(columns=['title','location','company_profile','description','requirements','employment_type','required_experience','industry','function'],inplace=True)

In [ ]:
Dataset01

,fraudulent,text
0,0,"Marketing Intern US, NY, New York We're Food52..."
1,0,"Customer Service - Cloud Video Production NZ, ..."
2,0,"Commissioning Machinery Assistant (CMA) US, IA..."
3,0,"Account Executive - Washington DC US, DC, Wash..."
4,0,"Bill Review Manager US, FL, Fort Worth SpotSou..."
...,...,...
17875,0,"Account Director - Distribution CA, ON, Toron..."
17876,0,"Payroll Accountant US, PA, Philadelphia WebLin..."
17877,0,Project Cost Control Staff Engineer - Cost Con...
17878,0,"Graphic Designer NG, LA, Lagos Nemsia Studios..."


Luego vamos a reemplazar los saltos de línea, salto de linea y retorno, y los espacios que son tabs

In [ ]:
Dataset01['text'] = Dataset01['text'].str.replace('\n', ' ')
Dataset01['text'] = Dataset01['text'].str.replace('\r', ' ')
Dataset01['text'] = Dataset01['text'].str.replace('\t', ' ')

Ahora, vamos a remover los números y los caracteres especiales

In [ ]:
Dataset01['text'] = Dataset01['text'].apply(lambda x: re.sub(r'[0-9]',' ',x))
Dataset01['text'] = Dataset01['text'].apply(lambda x: re.sub(r'[/(){}\[\]\|@,;.:-]',' ',x))

Y colocar todo el texto en minúsculas

In [ ]:
Dataset01['text']= Dataset01['text'].apply(lambda s:s.lower() if type(s) == str else s)

Después, comprobaremos que son strings, en caso de que si se divide el string en una lista de palabras

In [ ]:
Dataset01['text']= Dataset01['text'].apply(lambda s:" ".join(s.split()) if type(s) == str else s)

In [ ]:
Dataset01

,fraudulent,text
0,0,Marketing Intern US NY New York We're Food and...
1,0,Customer Service Cloud Video Production NZ Auc...
2,0,Commissioning Machinery Assistant CMA US IA We...
3,0,Account Executive Washington DC US DC Washingt...
4,0,Bill Review Manager US FL Fort Worth SpotSourc...
...,...,...
17875,0,Account Director Distribution CA ON Toronto Ve...
17876,0,Payroll Accountant US PA Philadelphia WebLinc ...
17877,0,Project Cost Control Staff Engineer Cost Contr...
17878,0,Graphic Designer NG LA Lagos Nemsia Studios is...
